In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, classification_report

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 KB 8.8 MB/s eta 0:00:00


In [ ]:
# !pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import lightgbm as lgb

In [ ]:
# Model and performance evaluation
from sklearn import preprocessing
from category_encoders import *
from sklearn.preprocessing import LabelEncoder
lbl = preprocessing.LabelEncoder()
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import precision_recall_fscore_support as score
# Hyperparameter tuning
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, STATUS_OK, space_eval

In [ ]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Crossroads/engg_data/crossroads_full_feature_set.csv")

In [ ]:
df.head(2)

,acct_id,acct_type_desc,event_name,event_date,plan_event_name,comp_name,section_name,row_name,SeatNum,price_code,PC1,Price,paid,add_datetime,class_name,status,Sales_Source,isHost,SeatType,TicketClass,Start Year,LastYear,Term,TicketType,SeatUniqueID,Season,ClubExpYear,Tenure,UniqueID,isAttended,Resold,ResalePrice,ResaleDate,isSTM,acct_type_group,plan_event_name_group,row_name_group,seat_nums_group,section_names_group,Zone,comp_names_group,price_group,PC1_group,class_name_grp,ticket_class_grp,TicketType_group,no_days_prebooked,gameday_minus_start_year,lastyear_minus_gameday,section_group,section_group_segment,section_name_segment,row_segment,seat_segment,Game_Type,Week,week_day,Game Time,Home Points,Away Points,unemployment_rate,covid_cases,Avg_Day_Temp,Precipitation,Avg_Dew_Point,weekend_flag
0,117948790,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,1,FR,F,111.0,Y,2021-03-31 16:08:52,OPEN,SOLD,NaN,0,GA,Manifest,NaN,NaN,NaN,Renewal,101-10-1,2021,NaN,25.0,CLT21HOU-101-10-1,Yes,NaN,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1
1,126665090,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,10,FR,F,111.0,Y,2021-03-31 16:08:19,OPEN,SOLD,NaN,0,GA,Manifest,NaN,NaN,NaN,Renewal,101-10-10,2021,NaN,20.0,CLT21HOU-101-10-10,No,NaN,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1


In [ ]:
def pivot(train_data, group_cols, agg_col):
    data = train_data.groupby(group_cols).agg({agg_col: "count"}).reset_index()
    data = pd.pivot_table(data, values=agg_col, index=[group_cols[0]], columns=[group_cols[1]], aggfunc=np.sum).reset_index().rename_axis(None, axis=1)
    data = data.fillna(0)
    data["count"] = data["Yes"] + data["No"]
    data["%Attended"] = 100*data["Yes"]/(data["Yes"]+data["No"])
    return data

In [ ]:
Seat_ID_df = pivot(df, ["SeatUniqueID", "isAttended"], "UniqueID")
Seat_ID_df.to_csv('seat_attendance.csv')

In [ ]:
accnt_type_df = pivot(df, ["acct_id", "isAttended"], "UniqueID")
accnt_type_df.head()

,acct_id,???,No,Yes,count,%Attended
0,190,6467.0,68700.0,369.0,69069.0,0.534248
1,14999990,0.0,8.0,0.0,8.0,0.000000
2,16999990,0.0,1.0,0.0,1.0,0.000000
3,110002390,14.0,29.0,97.0,126.0,76.984127
4,110003490,20.0,25.0,135.0,160.0,84.375000


In [ ]:
test_acct_ids = df[df["isAttended"]=="???"]["acct_id"].values

In [ ]:
temp = df[df["isAttended"]!="???"]
acct_count_df = temp.groupby("acct_id").agg({"event_name": "nunique"}).reset_index()
acct_count_df = acct_count_df.rename(columns={"event_name": "event_name_count"})
acct_count_df.head()

,acct_id,event_name_count
0,190,18
1,14999990,2
2,16999990,1
3,110002390,18
4,110003490,16


In [ ]:
"count", "%Attended", "event_name_count"

('count', '%Attended', 'event_name_count')

In [ ]:
df = df.merge(accnt_type_df[["acct_id", "count", "%Attended"]], on="acct_id", how="left")
df = df.merge(acct_count_df[["acct_id", "event_name_count"]], on="acct_id", how="left")

In [ ]:
df.isnull().sum()["%Attended"]

13443

In [ ]:
att  = df["isAttended"].values
df = df.drop("isAttended", 1)
df["isAttended"] = att

In [ ]:
df.head()

,acct_id,acct_type_desc,event_name,event_date,plan_event_name,comp_name,section_name,row_name,SeatNum,price_code,PC1,Price,paid,add_datetime,class_name,status,Sales_Source,isHost,SeatType,TicketClass,Start Year,LastYear,Term,TicketType,SeatUniqueID,Season,ClubExpYear,Tenure,UniqueID,Resold,ResalePrice,ResaleDate,isSTM,acct_type_group,plan_event_name_group,row_name_group,seat_nums_group,section_names_group,Zone,comp_names_group,price_group,PC1_group,class_name_grp,ticket_class_grp,TicketType_group,no_days_prebooked,gameday_minus_start_year,lastyear_minus_gameday,section_group,section_group_segment,section_name_segment,row_segment,seat_segment,Game_Type,Week,week_day,Game Time,Home Points,Away Points,unemployment_rate,covid_cases,Avg_Day_Temp,Precipitation,Avg_Dew_Point,weekend_flag,count,%Attended,event_name_count,isAttended
0,117948790,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,1,FR,F,111.0,Y,2021-03-31 16:08:52,OPEN,SOLD,NaN,0,GA,Manifest,NaN,NaN,NaN,Renewal,101-10-1,2021,NaN,25.0,CLT21HOU-101-10-1,NaN,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1,54.0,94.444444,18.0,Yes
1,126665090,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,10,FR,F,111.0,Y,2021-03-31 16:08:19,OPEN,SOLD,NaN,0,GA,Manifest,NaN,NaN,NaN,Renewal,101-10-10,2021,NaN,20.0,CLT21HOU-101-10-10,NaN,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1,72.0,70.833333,18.0,No
2,126665090,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,11,FR,F,111.0,Y,2021-03-31 16:08:19,OPEN,SOLD,NaN,0,GA,Manifest,NaN,NaN,NaN,Renewal,101-10-11,2021,NaN,20.0,CLT21HOU-101-10-11,NaN,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1,72.0,70.833333,18.0,No
3,1489599590,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,12,FR,F,111.0,Y,2021-03-31 16:10:20,OPEN,SOLD,NaN,0,GA,Manifest,NaN,NaN,NaN,Renewal,101-10-12,2021,NaN,4.0,CLT21HOU-101-10-12,NaN,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1,18.0,83.333333,18.0,Yes
4,115101990,Season,CLT21HOU,2021-10-17,21FS,Not Comp,101,10,13,FR,F,111.0,Y,2021-03-31 16:08:21,OPEN,SOLD,NaN,0,GA,Manifest,NaN,NaN,NaN,Renewal,101-10-13,2021,NaN,32.0,CLT21HOU-101-10-13,NaN,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,13:00,108,89,2.5,184.4,52.12,0.0,39.62,1,144.0,72.222222,18.0,No


In [ ]:
df = df.drop(["acct_id", "event_date", "section_name", "row_name", "SeatNum", "Game Time", 
              "add_datetime", "Start Year", "LastYear", "SeatUniqueID", "ResaleDate", 
              "ClubExpYear"], 1)


In [ ]:
df["Season"] = df["Season"].astype(str)

In [ ]:
cat_cols = []
for k,v in dict(df.dtypes).items():
  if v=="object":
    cat_cols.append(k)

cat_cols = list(set(cat_cols) - set(["event_name", "UniqueID", "isAttended"]))

In [ ]:
# cat_cols

In [ ]:
from pandas.core.indexes import category
for col in cat_cols:
  df[col] = df[col].astype("category")

In [ ]:
def label_encode(row, col):
  if row[col]=="Yes":
    return 1
  elif row[col]=="No":
    return 0
  else:
    return row[col]

df["isAttended"] = df.apply(lambda x: label_encode(x, "isAttended"), 1)

In [ ]:
df.shape

(1286916, 57)

In [ ]:
df.head()

,acct_type_desc,event_name,plan_event_name,comp_name,price_code,PC1,Price,paid,class_name,status,Sales_Source,isHost,SeatType,TicketClass,Term,TicketType,Season,Tenure,UniqueID,Resold,ResalePrice,isSTM,acct_type_group,plan_event_name_group,row_name_group,seat_nums_group,section_names_group,Zone,comp_names_group,price_group,PC1_group,class_name_grp,ticket_class_grp,TicketType_group,no_days_prebooked,gameday_minus_start_year,lastyear_minus_gameday,section_group,section_group_segment,section_name_segment,row_segment,seat_segment,Game_Type,Week,week_day,Home Points,Away Points,unemployment_rate,covid_cases,Avg_Day_Temp,Precipitation,Avg_Dew_Point,weekend_flag,count,%Attended,event_name_count,isAttended
0,Season,CLT21HOU,21FS,Not Comp,FR,F,111.0,Y,OPEN,SOLD,NaN,0,GA,Manifest,NaN,Renewal,2021,25.0,CLT21HOU-101-10-1,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.4,52.12,0.0,39.62,1,54.0,94.444444,18.0,1
1,Season,CLT21HOU,21FS,Not Comp,FR,F,111.0,Y,OPEN,SOLD,NaN,0,GA,Manifest,NaN,Renewal,2021,20.0,CLT21HOU-101-10-10,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.4,52.12,0.0,39.62,1,72.0,70.833333,18.0,0
2,Season,CLT21HOU,21FS,Not Comp,FR,F,111.0,Y,OPEN,SOLD,NaN,0,GA,Manifest,NaN,Renewal,2021,20.0,CLT21HOU-101-10-11,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.4,52.12,0.0,39.62,1,72.0,70.833333,18.0,0
3,Season,CLT21HOU,21FS,Not Comp,FR,F,111.0,Y,OPEN,SOLD,NaN,0,GA,Manifest,NaN,Renewal,2021,4.0,CLT21HOU-101-10-12,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.4,52.12,0.0,39.62,1,18.0,83.333333,18.0,1
4,Season,CLT21HOU,21FS,Not Comp,FR,F,111.0,Y,OPEN,SOLD,NaN,0,GA,Manifest,NaN,Renewal,2021,32.0,CLT21HOU-101-10-13,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.4,52.12,0.0,39.62,1,144.0,72.222222,18.0,0


In [ ]:
train_df = df[df["isAttended"]!="???"]
test_df = df[df["isAttended"]=="???"]

In [ ]:
val_data = pd.read_csv("/content/drive/MyDrive/Crossroads/Ensemble_Model/Ensemble_Validation_DataSet_v1")
val_data = val_data.drop("Unnamed: 0", 1)
val_df = train_df[train_df["UniqueID"].isin(val_data["UniqueID"].values.tolist())]

In [ ]:
train_df = train_df[~train_df["UniqueID"].isin(val_df["UniqueID"].values.tolist())]

In [ ]:
train_cnt = train_df["count"].values
train_att = train_df["%Attended"].values
train_event_cnt = train_df["event_name_count"].values

val_cnt = val_df["count"].values
val_att = val_df["%Attended"].values
val_event_cnt = val_df["event_name_count"].values

test_cnt = test_df["count"].values
test_att = test_df["%Attended"].values
test_event_cnt = test_df["event_name_count"].values

In [ ]:
train_df = train_df.drop(["count"], 1)
val_df = val_df.drop(["count"], 1)
test_df = test_df.drop(["count"], 1)

In [ ]:
train_df.head()

,acct_type_desc,event_name,plan_event_name,comp_name,price_code,PC1,Price,paid,class_name,status,Sales_Source,isHost,SeatType,TicketClass,Term,TicketType,Season,Tenure,UniqueID,Resold,ResalePrice,isSTM,acct_type_group,plan_event_name_group,row_name_group,seat_nums_group,section_names_group,Zone,comp_names_group,price_group,PC1_group,class_name_grp,ticket_class_grp,TicketType_group,no_days_prebooked,gameday_minus_start_year,lastyear_minus_gameday,section_group,section_group_segment,section_name_segment,row_segment,seat_segment,Game_Type,Week,week_day,Home Points,Away Points,unemployment_rate,covid_cases,Avg_Day_Temp,Precipitation,Avg_Dew_Point,weekend_flag,%Attended,event_name_count,isAttended
0,Season,CLT21HOU,21FS,Not Comp,FR,F,111.0,Y,OPEN,SOLD,NaN,0,GA,Manifest,NaN,Renewal,2021,25.0,CLT21HOU-101-10-1,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.4,52.12,0.0,39.62,1,94.444444,18.0,1
1,Season,CLT21HOU,21FS,Not Comp,FR,F,111.0,Y,OPEN,SOLD,NaN,0,GA,Manifest,NaN,Renewal,2021,20.0,CLT21HOU-101-10-10,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.4,52.12,0.0,39.62,1,70.833333,18.0,0
2,Season,CLT21HOU,21FS,Not Comp,FR,F,111.0,Y,OPEN,SOLD,NaN,0,GA,Manifest,NaN,Renewal,2021,20.0,CLT21HOU-101-10-11,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.4,52.12,0.0,39.62,1,70.833333,18.0,0
3,Season,CLT21HOU,21FS,Not Comp,FR,F,111.0,Y,OPEN,SOLD,NaN,0,GA,Manifest,NaN,Renewal,2021,4.0,CLT21HOU-101-10-12,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.4,52.12,0.0,39.62,1,83.333333,18.0,1
6,Season,CLT21HOU,21FS,Not Comp,FR,F,111.0,Y,OPEN,SOLD,NaN,0,GA,Manifest,NaN,Renewal,2021,32.0,CLT21HOU-101-10-15,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.4,52.12,0.0,39.62,1,72.222222,18.0,0


In [ ]:
test_df.head()

,acct_type_desc,event_name,plan_event_name,comp_name,price_code,PC1,Price,paid,class_name,status,Sales_Source,isHost,SeatType,TicketClass,Term,TicketType,Season,Tenure,UniqueID,Resold,ResalePrice,isSTM,acct_type_group,plan_event_name_group,row_name_group,seat_nums_group,section_names_group,Zone,comp_names_group,price_group,PC1_group,class_name_grp,ticket_class_grp,TicketType_group,no_days_prebooked,gameday_minus_start_year,lastyear_minus_gameday,section_group,section_group_segment,section_name_segment,row_segment,seat_segment,Game_Type,Week,week_day,Home Points,Away Points,unemployment_rate,covid_cases,Avg_Day_Temp,Precipitation,Avg_Dew_Point,weekend_flag,%Attended,event_name_count,isAttended
193038,Season,CLT21LV,21FS,Not Comp,FR,F,135.0,Y,OPEN,SOLD,NaN,0,GA,Manifest,NaN,Renewal,2021,25.0,CLT21LV-101-10-1,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,276.2,NaN,NaN,1.0,High,High,High,High,Reg,17,Sunday,420,316,2.2,1761.0,29.27,0.0,24.77,1,94.444444,18.0,???
193039,Season,CLT21LV,21FS,Not Comp,FR,F,135.0,Y,OPEN,SOLD,NaN,0,GA,Manifest,NaN,Renewal,2021,20.0,CLT21LV-101-10-10,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,276.2,NaN,NaN,1.0,High,High,High,High,Reg,17,Sunday,420,316,2.2,1761.0,29.27,0.0,24.77,1,70.833333,18.0,???
193040,Season,CLT21LV,21FS,Not Comp,FR,F,135.0,Y,OPEN,SOLD,NaN,0,GA,Manifest,NaN,Renewal,2021,20.0,CLT21LV-101-10-11,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,276.2,NaN,NaN,1.0,High,High,High,High,Reg,17,Sunday,420,316,2.2,1761.0,29.27,0.0,24.77,1,70.833333,18.0,???
193041,Season,CLT21LV,21FS,Not Comp,FR,F,135.0,Y,OPEN,SOLD,NaN,0,GA,Manifest,NaN,Renewal,2021,4.0,CLT21LV-101-10-12,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,276.2,NaN,NaN,1.0,High,High,High,High,Reg,17,Sunday,420,316,2.2,1761.0,29.27,0.0,24.77,1,83.333333,18.0,???
193042,Season,CLT21LV,21FS,Not Comp,FR,F,135.0,Y,OPEN,SOLD,NaN,0,GA,Manifest,NaN,Renewal,2021,32.0,CLT21LV-101-10-13,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,276.2,NaN,NaN,1.0,High,High,High,High,Reg,17,Sunday,420,316,2.2,1761.0,29.27,0.0,24.77,1,72.222222,18.0,???


In [ ]:
test_df["isAttended"].unique()

array(['???'], dtype=object)

In [ ]:
train_df.shape, val_df.shape, test_df.shape

((926582, 56), (231646, 56), (128688, 56))

In [ ]:
train_df["event_name"].nunique(), val_df["event_name"].nunique(), test_df["event_name"].nunique()

(18, 18, 2)

In [ ]:
X_train = train_df.iloc[:, :-1]
y_train = train_df.iloc[:, -1].values
X_val = val_df.iloc[:, :-1]
y_val = val_df.iloc[:, -1].values
X_test = test_df.iloc[:, :-1]
y_test = test_df.iloc[:, -1].values

In [ ]:
enc = TargetEncoder(cols=cat_cols, min_samples_leaf=20, smoothing=10, handle_missing = 'return_nan').fit(X_train, y_train)
X_train_encoded = enc.transform(X_train)
X_val_encoded = enc.transform(X_val)
X_test_encoded = enc.transform(X_test)

In [ ]:
X_train_encoded.head()

,acct_type_desc,event_name,plan_event_name,comp_name,price_code,PC1,Price,paid,class_name,status,Sales_Source,isHost,SeatType,TicketClass,Term,TicketType,Season,Tenure,UniqueID,Resold,ResalePrice,isSTM,acct_type_group,plan_event_name_group,row_name_group,seat_nums_group,section_names_group,Zone,comp_names_group,price_group,PC1_group,class_name_grp,ticket_class_grp,TicketType_group,no_days_prebooked,gameday_minus_start_year,lastyear_minus_gameday,section_group,section_group_segment,section_name_segment,row_segment,seat_segment,Game_Type,Week,week_day,Home Points,Away Points,unemployment_rate,covid_cases,Avg_Day_Temp,Precipitation,Avg_Dew_Point,weekend_flag,%Attended,event_name_count
0,0.847217,CLT21HOU,0.831096,0.841971,0.856563,0.846325,111.0,0.83152,0.828021,0.841971,NaN,0,0.773091,0.799824,NaN,0.848955,0.752518,25.0,CLT21HOU-101-10-1,NaN,NaN,NaN,0.845861,0.833541,0.801652,0.78405,0.814664,0.754976,0.840094,0.841979,0.833354,0.833235,0.797324,0.843415,199.4,NaN,NaN,1.0,0.67198,0.684206,0.677035,0.680039,0.817374,6,0.782799,108,89,2.5,184.4,52.12,0.0,39.62,1,94.444444,18.0
1,0.847217,CLT21HOU,0.831096,0.841971,0.856563,0.846325,111.0,0.83152,0.828021,0.841971,NaN,0,0.773091,0.799824,NaN,0.848955,0.752518,20.0,CLT21HOU-101-10-10,NaN,NaN,NaN,0.845861,0.833541,0.801652,0.78405,0.814664,0.754976,0.840094,0.841979,0.833354,0.833235,0.797324,0.843415,199.4,NaN,NaN,1.0,0.67198,0.684206,0.677035,0.680039,0.817374,6,0.782799,108,89,2.5,184.4,52.12,0.0,39.62,1,70.833333,18.0
2,0.847217,CLT21HOU,0.831096,0.841971,0.856563,0.846325,111.0,0.83152,0.828021,0.841971,NaN,0,0.773091,0.799824,NaN,0.848955,0.752518,20.0,CLT21HOU-101-10-11,NaN,NaN,NaN,0.845861,0.833541,0.801652,0.78405,0.814664,0.754976,0.840094,0.841979,0.833354,0.833235,0.797324,0.843415,199.4,NaN,NaN,1.0,0.67198,0.684206,0.677035,0.680039,0.817374,6,0.782799,108,89,2.5,184.4,52.12,0.0,39.62,1,70.833333,18.0
3,0.847217,CLT21HOU,0.831096,0.841971,0.856563,0.846325,111.0,0.83152,0.828021,0.841971,NaN,0,0.773091,0.799824,NaN,0.848955,0.752518,4.0,CLT21HOU-101-10-12,NaN,NaN,NaN,0.845861,0.833541,0.801652,0.78405,0.814664,0.754976,0.840094,0.841979,0.833354,0.833235,0.797324,0.843415,199.4,NaN,NaN,1.0,0.67198,0.684206,0.677035,0.680039,0.817374,6,0.782799,108,89,2.5,184.4,52.12,0.0,39.62,1,83.333333,18.0
6,0.847217,CLT21HOU,0.831096,0.841971,0.856563,0.846325,111.0,0.83152,0.828021,0.841971,NaN,0,0.773091,0.799824,NaN,0.848955,0.752518,32.0,CLT21HOU-101-10-15,NaN,NaN,NaN,0.845861,0.833541,0.801652,0.78405,0.814664,0.754976,0.840094,0.841979,0.833354,0.833235,0.797324,0.843415,199.4,NaN,NaN,1.0,0.67198,0.684206,0.677035,0.680039,0.817374,6,0.782799,108,89,2.5,184.4,52.12,0.0,39.62,1,72.222222,18.0


In [ ]:
train_ids = X_train["UniqueID"].values
val_ids = X_val["UniqueID"].values
test_ids = X_test["UniqueID"].values

X_train = X_train.drop(["UniqueID", "event_name"], 1)
X_val = X_val.drop(["UniqueID", "event_name"], 1)
X_test = X_test.drop(["UniqueID", "event_name"], 1)

In [ ]:
train_ids = X_train_encoded["UniqueID"].values
val_ids = X_val_encoded["UniqueID"].values
test_ids = X_test_encoded["UniqueID"].values

X_train_encoded = X_train_encoded.drop(["UniqueID", "event_name"], 1)
X_val_encoded = X_val_encoded.drop(["UniqueID", "event_name"], 1)
X_test_encoded = X_test_encoded.drop(["UniqueID", "event_name"], 1)

In [ ]:
X_train.head()

,acct_type_desc,plan_event_name,comp_name,price_code,PC1,Price,paid,class_name,status,Sales_Source,isHost,SeatType,TicketClass,Term,TicketType,Season,Tenure,Resold,ResalePrice,isSTM,acct_type_group,plan_event_name_group,row_name_group,seat_nums_group,section_names_group,Zone,comp_names_group,price_group,PC1_group,class_name_grp,ticket_class_grp,TicketType_group,no_days_prebooked,gameday_minus_start_year,lastyear_minus_gameday,section_group,section_group_segment,section_name_segment,row_segment,seat_segment,Game_Type,Week,week_day,Home Points,Away Points,unemployment_rate,covid_cases,Avg_Day_Temp,Precipitation,Avg_Dew_Point,weekend_flag,%Attended,event_name_count
0,Season,21FS,Not Comp,FR,F,111.0,Y,OPEN,SOLD,NaN,0,GA,Manifest,NaN,Renewal,2021,25.0,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.4,52.12,0.0,39.62,1,94.444444,18.0
1,Season,21FS,Not Comp,FR,F,111.0,Y,OPEN,SOLD,NaN,0,GA,Manifest,NaN,Renewal,2021,20.0,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.4,52.12,0.0,39.62,1,70.833333,18.0
2,Season,21FS,Not Comp,FR,F,111.0,Y,OPEN,SOLD,NaN,0,GA,Manifest,NaN,Renewal,2021,20.0,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.4,52.12,0.0,39.62,1,70.833333,18.0
3,Season,21FS,Not Comp,FR,F,111.0,Y,OPEN,SOLD,NaN,0,GA,Manifest,NaN,Renewal,2021,4.0,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.4,52.12,0.0,39.62,1,83.333333,18.0
6,Season,21FS,Not Comp,FR,F,111.0,Y,OPEN,SOLD,NaN,0,GA,Manifest,NaN,Renewal,2021,32.0,NaN,NaN,NaN,G1,G1,G1,G1,G1,N,G1,price_group_3,PC1_group_2,class_name_2,ticket_class_1,group_0,199.4,NaN,NaN,1.0,Low,Low,Low,Low,Reg,6,Sunday,108,89,2.5,184.4,52.12,0.0,39.62,1,72.222222,18.0


In [ ]:
df["isAttended"].value_counts()

1      900945
0      257283
???    128688
Name: isAttended, dtype: int64

In [ ]:
257283/900945, 900945/257283

(0.28557015134109187, 3.5017665372372058)

In [ ]:
def learning_rate_010_decay_power_099(current_iter):
    base_learning_rate = 0.1
    lr = base_learning_rate  * np.power(.99, current_iter)
    return lr if lr > 1e-3 else 1e-3

def learning_rate_010_decay_power_0995(current_iter):
    base_learning_rate = 0.1
    lr = base_learning_rate  * np.power(.995, current_iter)
    return lr if lr > 1e-3 else 1e-3

def learning_rate_005_decay_power_099(current_iter):
    base_learning_rate = 0.05
    lr = base_learning_rate  * np.power(.99, current_iter)
    return lr if lr > 1e-3 else 1e-3

In [ ]:
n_HP_points_to_test = 50

In [ ]:
fit_params={"early_stopping_rounds":30, 
            "eval_metric" : 'binary_logloss', 
            "eval_set" : [(X_val_encoded,[int(each) for each in y_val])],
            'eval_names': ['valid'],
            #'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_099)],
            'verbose': 100,
            'categorical_feature': 'auto'}

In [ ]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
# param_test ={'num_leaves': sp_randint(6, 30), 
#              'min_child_samples': sp_randint(100, 500), 
#              'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
#              'subsample': sp_uniform(loc=0.2, scale=0.8), 
#              'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
#             #  'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
#              'reg_lambda': [5, 10, 20, 50, 100]}

In [ ]:
# clf = lgb.LGBMClassifier(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=5000)
# gs = RandomizedSearchCV(
#     estimator=clf, param_distributions=param_test, 
#     n_iter=n_HP_points_to_test,
#     scoring='roc_auc',
#     cv=3,
#     refit=True,
#     random_state=314,
#     verbose=True)

In [ ]:
# gs.fit(X_train_encoded, [int(each) for each in y_train], **fit_params)
# print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))

In [ ]:
#Best params after hyperparameter tuning
get_params_ = {'boosting_type': 'gbdt',
 'class_weight': None,
 'num_leaves': 8,
 'colsample_bytree': 0.6579,
 'importance_type': 'split',
 'learning_rate': 0.2,
 'max_depth': -1,
 'min_child_samples': 322,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 10000,
 'n_jobs': 4,
 'num_leaves': 31,
 'objective': None,
 'random_state': 314,
 'reg_alpha': 0.0,
 'reg_lambda': 50.0,
 'silent': True,
 'subsample': 0.9274,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'metric': 'None'}

In [ ]:
# Best score reached: 0.7947780130028633 with params: {'colsample_bytree': 0.6578931113743162, 'min_child_samples': 322, 'min_child_weight': 0.001, 'num_leaves': 8, 'reg_lambda': 50, 'subsample': 0.9274016358891894} 


In [ ]:
opt_parameters = {'colsample_bytree': 0.6579, 'min_child_samples': 322, 'min_child_weight': 0.001, 
                  'num_leaves': 8, 'reg_alpha': 0, 'reg_lambda': 50, 'subsample': 0.9274} 


In [ ]:
# clf_sw = lgb.LGBMClassifier(**get_params_)
#set optimal parameters
# clf_sw.set_params(**opt_parameters)

In [ ]:
# gs_sample_weight = GridSearchCV(estimator=clf_sw, 
#                                 param_grid={'scale_pos_weight':[1,2,6,12]},
#                                 scoring='roc_auc',
#                                 cv=5,
#                                 refit=True,
#                                 verbose=True)

In [ ]:
# gs_sample_weight.fit(X_train_encoded, [int(each) for each in y_train], **fit_params)
# print('Best score reached: {} with params: {} '.format(gs_sample_weight.best_score_, gs_sample_weight.best_params_))

In [ ]:
# print("Valid+-Std     Train  :   Parameters")
# for i in np.argsort(gs_sample_weight.cv_results_['mean_test_score'])[-5:]:
#     print('{1:.3f}+-{3:.3f}     :  {0}'.format(gs_sample_weight.cv_results_['params'][i], 
#                                     gs_sample_weight.cv_results_['mean_test_score'][i], 
#                                     gs_sample_weight.cv_results_['std_test_score'][i]))

In [ ]:
# print('Best score reached: {} with params: {} '.format(gs_sample_weight.best_score_, gs_sample_weight.best_params_))

In [ ]:
# opt_sw_params = gs_sample_weight.best_params_

In [ ]:
# del clf_final
import gc 
gc.collect()

2135

In [ ]:
clf_final = lgb.LGBMClassifier(**get_params_)


In [ ]:
# clf_final = lgb.LGBMClassifier(**clf.get_params())
clf_final = lgb.LGBMClassifier(**get_params_)
#set optimal parameters
# clf_final.set_params(**opt_sw_params)

#Train the final model with learning rate decay
clf_final.fit(X_train_encoded, [int(each) for each in y_train], **fit_params, callbacks=[lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_0995)])


Training until validation scores don't improve for 30 rounds.
[100]	valid's binary_logloss: 0.301052
[200]	valid's binary_logloss: 0.299783
[300]	valid's binary_logloss: 0.299304
[400]	valid's binary_logloss: 0.299021
[500]	valid's binary_logloss: 0.29888
[600]	valid's binary_logloss: 0.298804
[700]	valid's binary_logloss: 0.298755
[800]	valid's binary_logloss: 0.298727
[900]	valid's binary_logloss: 0.298707
[1000]	valid's binary_logloss: 0.298693
[1100]	valid's binary_logloss: 0.298679
[1200]	valid's binary_logloss: 0.298665
[1300]	valid's binary_logloss: 0.298651
[1400]	valid's binary_logloss: 0.298637
[1500]	valid's binary_logloss: 0.298623
[1600]	valid's binary_logloss: 0.29861
[1700]	valid's binary_logloss: 0.298595
[1800]	valid's binary_logloss: 0.298582
[1900]	valid's binary_logloss: 0.298568
[2000]	valid's binary_logloss: 0.298556
[2100]	valid's binary_logloss: 0.298544
[2200]	valid's binary_logloss: 0.298532
[2300]	valid's binary_logloss: 0.29852
[2400]	valid's binary_logloss:

LGBMClassifier(colsample_bytree=0.6579, learning_rate=0.2, metric='None',
               min_child_samples=322, n_estimators=10000, n_jobs=4,
               random_state=314, reg_lambda=50.0, subsample=0.9274)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf_final, X_train_encoded, [int(each) for each in y_train], cv=4)
scores


array([0.87938061, 0.64310629, 0.84913553, 0.61703037])

In [ ]:
y=[int(each) for each in y_train]

In [ ]:
y_pred_val_logits = clf_final.predict_proba(X_val_encoded)

In [ ]:
y_pred_val_logits = [each[1] for each in y_pred_val_logits]

In [ ]:
y_pred_val = [1 if each>threshold else 0 for each in y_pred_val_logits]

print(classification_report([int(each) for each in y_val.tolist()], y_pred_val))

              precision    recall  f1-score   support

           0       0.85      0.56      0.67     51457
           1       0.88      0.97      0.93    180189

    accuracy                           0.88    231646
   macro avg       0.87      0.76      0.80    231646
weighted avg       0.88      0.88      0.87    231646



In [ ]:
len(y_pred_val) - sum(y_pred_val), len(y_pred_val)

(33775, 231646)

In [ ]:
val_sub_df = pd.DataFrame()
val_sub_df["UniqueID"] = val_ids
val_sub_df["XGB_model_preds"] = y_pred_val_logits

In [ ]:
y_pred_val = [each[1] for each in clf_final.predict_proba(X_val_encoded)]

In [ ]:
y_pred_val = [1 if each>threshold else 0 for each in y_pred_val]

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix([int(each) for each in y_val.tolist()], y_pred_val))

[[ 28610  22847]
 [  5165 175024]]


In [ ]:
test_pred_logits = [each[1] for each in clf_final.predict_proba(X_test_encoded)]

In [ ]:
test_preds = [1 if each>threshold else 0 for each in test_pred_logits]

In [ ]:
test_sub_df = pd.DataFrame()
test_sub_df["UniqueID"] = test_ids
test_sub_df["XGB_model_preds"] = test_pred_logits

In [ ]:
1 - (sum(test_preds)/len(test_preds))

0.1825655849807286

In [ ]:
len(test_preds) - sum(test_preds), len(test_preds)

(23494, 128688)

In [ ]:
len(test_acct_ids)

128688

In [ ]:
val_sub_df.to_csv("/content/drive/MyDrive/Crossroads/Ensemble_Model/saved_preds/LGB_v1.2_TE_Acc_ID_val.csv", index=False)

In [ ]:
test_sub_df.to_csv("/content/drive/MyDrive/Crossroads/Ensemble_Model/saved_preds/LGB_v1.2_TE_Acc_ID_test.csv", index=False)